# Automated ML

we start by importing all the dependencies that we will need to complete the project.

In [10]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails


## Dataset

### Overview
We first of all define the workspace and create the experiment 'Parkinson-classification-AutoML' (config.json in the same directory)

We then access to the dataset from the external link (placed in the github)

To  prepare the data(read the data using TabularDatasetFactory  then transforme it to a dataframe then store it in a csv file ) and send it to the default datastore so that we can use it in the Automl Config , to finally create the compute cluster 

The dataset used in this notebook  is composed of a range of biomedical voice measurements from 31 people, 23 with Parkinson's disease (PD). Each column in the data is a particular voice measure, and each row corresponds one of 195 voice recording from these individuals ("name" column)

In [2]:
#Connection to the workspace and definition of the experiment 
ws = Workspace.from_config()
experiment_name = 'Parkinson-classification-AutoML'

experiment=Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-132562
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-132562


In [3]:
#Get the data from github
path_url = 'https://raw.githubusercontent.com/hananeouhammouch/Parkinsons-detection/master/parkinsons.data'
ds = TabularDatasetFactory.from_delimited_files(path = path_url)
ds.to_pandas_dataframe().head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [5]:
#Prepare the data and send it to the datastore
ds.to_pandas_dataframe().to_csv("./training_dataset.csv")
datastore = ws.get_default_datastore()
datastore.upload(src_dir = "./", target_path = "data/")
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_dataset.csv"))])


Uploading an estimated of 5 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 5
Uploading ./config.json
Uploaded ./config.json, 2 files out of an estimated total of 5
Uploading ./training_dataset.csv
Uploaded ./training_dataset.csv, 3 files out of an estimated total of 5
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-30-23-29-16.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-30-23-29-16.ipynb, 4 files out of an estimated total of 5
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 5 files out of an estimated total of 5
Uploaded 5 files


In [6]:
#Creation of the compute-cluster
amlcompute_cluster_name = "cpu-cluster"

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=5)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

Creating
Succeeded....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

Bellow the description of the configuration used in Automl


|Setting |Why?|
|-|-|
|**experiment_timeout_minutes**|Maximum amount of time in minutes that all iterations combined can take before the experiment terminates (15 minute the dataset include only 195 line)|
|**max_concurrent_iterations**|To help manage child runs and when they can be performed, we create a dedicated cluster per experiment, and match the number of this setting (4) to the number of nodes in the cluster(5-1))|

In [7]:
# Automl settings here
automl_settings = {
    "experiment_timeout_minutes" :15,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}

# Automl config here
automl_config = AutoMLConfig(
    task="classification",
    compute_target=aml_compute,
    training_data=training_data,
    label_column_name="status",
    **automl_settings
)

In [8]:
# TODO: Submit your experiment
auto_ml_run = experiment.submit(config = automl_config, show_output = True)


Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_a0240568-0f51-4765-ab12-51ce97260977

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values we

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(auto_ml_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Oarkinson-detection-automl', model_path='./')

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service